In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

In [ ]:

#!conda install -y s3fs

import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role

##The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
#The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Here we use the get_execution_role function to obtain the role arn which was specified when creating the notebook.
bucket = 'data'
prefix = 'rnd'

sagemaker_session = sagemaker.Session()
role = get_execution_role()

s3_data_path = "{}/{}/2013-8/".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [ ]:
all_files = glob.glob(os.path.join(s3_data_path, "1*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, sep = ';\t') for f in all_files)
concatenated_df   = pd.concat(df_from_each_file)

In [ ]:
concatenated_df['Timestamp'] = pd.to_datetime(concatenated_df['Timestamp [ms]'], unit = 's')
concatenated_df.apply(pd.to_numeric, errors='ignore')
concatenated_df.describe()
concatenated_df['weekday'] = concatenated_df['Timestamp'].dt.dayofweek


concatenated_df['weekend'] = ((concatenated_df.weekday) // 5 == 1).astype(float)
# Feature engineering with the date
concatenated_df['month']=concatenated_df.Timestamp.dt.month 
concatenated_df['day']=concatenated_df.Timestamp.dt.day
concatenated_df.set_index('Timestamp',inplace=True)
concatenated_df["CPU usage prev"] = concatenated_df['CPU usage [%]'].shift(1)
concatenated_df["CPU_diff"] = concatenated_df['CPU usage [%]'] - concatenated_df["CPU usage prev"]
concatenated_df["received_prev"] = concatenated_df['Network received throughput [KB/s]'].shift(1)
concatenated_df["received_diff"] = concatenated_df['Network received throughput [KB/s]']- concatenated_df["received_prev"]

concatenated_df["transmitted_prev"] = concatenated_df['Network transmitted throughput [KB/s]'].shift(1)
concatenated_df["transmitted_diff"] = concatenated_df['Network transmitted throughput [KB/s]']- concatenated_df["transmitted_prev"]

cpu = concatenated_df[['CPU usage [MHZ]']]
receive = concatenated_df[['Network received throughput [KB/s]']]
transmit = concatenated_df[['Network transmitted throughput [KB/s]']]
#concatenated_df.drop('CPU capacity provisioned [MHZ]', axis=1, inplace=True)
#concatenated_df.drop('Memory capacity provisioned [KB]', axis=1, inplace=True)
concatenated_df.drop('Timestamp [ms]', axis=1, inplace=True) 
#concatenated_df.drop('Memory usage [KB]', axis=1, inplace=True)
provisioned = concatenated_df[['CPU capacity provisioned [MHZ]']]

moving_average(cpu, 24) # prediction for the last observed day (past 24 hours)
hourlydat = concatenated_df.resample('H').sum()
#hourlydat.iloc[:,5:7].plot(style=[':', '--', '-'], figsize=(20,10), linewidth=3, fontsize=20)
#plt.ylabel('Resampled Data');
hourlycpu = cpu.resample('H').sum()
hourlytransmit = transmit.resample('H').sum()
hourlyreceive = receive.resample('H').sum()




In [ ]:
hourlydat["start"] = hourlydat.index

In [ ]:
hourlydat['target'] = hourlydat['CPU usage [MHZ]']

jsondat=pd.DataFrame(hourlydat['start'])

jsondat['start'] = hourlydat.index

result = pd.concat([hourlydat['start'], hourlydat['target']], axis=1, sort=False)

result = result.reset_index()

result.drop('Timestamp', axis=1, inplace=True)

result.start = result.start.astype(str)

#result.set_index('start',inplace=True)


result.head()